In [1]:
import csv
from collections import defaultdict
from fuzzywuzzy import fuzz

# Constants representing the paths to the datasets
offer_retailer = 'offer_retailer.csv'
brand_category= 'brand_category.csv'
categories = 'categories.csv'


In [2]:
class OfferSearchTool:
    def __init__(self):
        self.offers = []
        self.brands = {}
        self.categories = {}
        self.load_datasets()

    def load_datasets(self):
        # Load the offer dataset
        with open(offer_retailer, 'r') as file:
            reader = csv.DictReader(file)
            for row in reader:
                self.offers.append(row)

        # Load the brand and category datasets
        with open(brand_category, 'r') as file:
            reader = csv.DictReader(file)
            for row in reader:
                self.brands[row['BRAND']] = row['BRAND_BELONGS_TO_CATEGORY']

        with open(categories, 'r') as file:
            reader = csv.DictReader(file)
            for row in reader:
                self.categories[row['PRODUCT_CATEGORY']] = row['IS_CHILD_CATEGORY_TO']


    def search(self, query):
    # Perform search using fuzzy matching
        results = defaultdict(float)
        for offer in self.offers:
        # Determine the type of query and adjust the search accordingly
            if query.lower() in self.categories:
            # Query is a category
                category_match_score = self.score_similarity(query, self.brands.get(offer['BRAND'], ''))
                results[offer['OFFER']] += category_match_score
            elif query.lower() in self.brands:
            # Query is a brand
                brand_match_score = self.score_similarity(query, offer['BRAND'])
                results[offer['OFFER']] += brand_match_score
            else:
            # Query is potentially a retailer
                retailer_match_score = self.score_similarity(query, offer['RETAILER'])
                results[offer['OFFER']] += retailer_match_score

        sorted_results = sorted(results.items(), key=lambda item: item[1], reverse=True)
        return sorted_results

    def score_similarity(self, query, data_point):
        # Use fuzzy string matching for the similarity score
        return fuzz.partial_ratio(query.lower(), data_point.lower())

    def run_cli(self):
        # Command-line interface for the search tool
        while True:
            query = input("Enter your search query (type 'exit' to stop): ")
            if query.lower() == 'exit':
                break
            results = self.search(query)
            if not results:
                print("No results found.")
                continue
            for offer, score in results:
                print(f"Offer: {offer}, Score: {score}")


if __name__ == "__main__":
    tool = OfferSearchTool()
    tool.run_cli()


Enter your search query (type 'exit' to stop): diapers
Offer: Spend $210 at Randalls, Score: 86.0
Offer: GATORADE® Fast Twitch®, 12-ounce single serve, buy 2 at Kroger, Score: 86.0
Offer: Any Randalls receipt, Score: 86.0
Offer: Shop 2 times at Randalls, Score: 86.0
Offer: Gorton's, at select retailers, Score: 58.0
Offer: Reese's Chocolate Cones, 8 count at GIANT OR MARTIN’S, Score: 58.0
Offer: Reese's Peanut Butter Bar, 6 count, at GIANT OR MARTIN’S, Score: 58.0
Offer: Gorton's at select retailers, Score: 58.0
Offer: Dr Pepper®, Regular OR Zero Sugar Strawberries and Cream 12 pack, at select retailers, Score: 57.0
Offer: Butterball, select varieties, spend $10 at King Soopers, Score: 57.0
Offer: Any Albertsons receipt, Score: 57.0
Offer: Shop 2 times at Albertsons, Score: 57.0
Offer: Spend $65 at Albertsons, Score: 57.0
Offer: Bubbies Pickles, Sauerkraut, and Condiments at Whole Foods or Sprouts, Score: 57.0
Offer: Sign up for McAlister's Deli Rewards, tap for details, Score: 57.0
Off

Enter your search query (type 'exit' to stop): Huggies
Offer: Reese's Peanut Butter Bar, 6 count, at GIANT OR MARTIN’S, Score: 72.0
Offer: Kradle, select varieties, at Pet Supplies Plus, Score: 57.0
Offer: Gorton's, at select retailers, Score: 43.0
Offer: Spend $15 at Nekter, Score: 43.0
Offer: Spend $210 at Randalls, Score: 43.0
Offer: Sign up for The Club Card or The Club+ Card full-priced membership* (New Members Only), Score: 43.0
Offer: Sign up for McAlister's Deli Rewards, tap for details, Score: 43.0
Offer: GATORADE® Fast Twitch®, 12-ounce single serve, buy 2 at Kroger, Score: 43.0
Offer: Any Randalls receipt, Score: 43.0
Offer: Shop 2 times at Randalls, Score: 43.0
Offer: Spend $10 at Nekter, Score: 43.0
Offer: Spend $15 at Burger King, Score: 43.0
Offer: Beyond Steak™ Plant-Based seared tips, 10 ounce at Target, Score: 33.0
Offer: L'Oréal Paris Hair color, select varieties, spend $19 at Target, Score: 33.0
Offer: L'Oréal Paris Makeup, spend $35 at Target, Score: 33.0
Offer: Ba

Enter your search query (type 'exit' to stop): Target
Offer: Beyond Steak™ Plant-Based seared tips, 10 ounce at Target, Score: 100.0
Offer: L'Oréal Paris Hair color, select varieties, spend $19 at Target, Score: 100.0
Offer: L'Oréal Paris Makeup, spend $35 at Target, Score: 100.0
Offer: Back to the Roots Grow Seed Starting Pots OR Germination Trays, at Walmart or Target, Score: 100.0
Offer: Dove Hand Wash, select varieties, buy 2 at Target, Score: 100.0
Offer: Durex® Massage & Play 2 in 1 Pleasure Gel, select varieties, at Walmart or Target, Score: 100.0
Offer: Welch's® Juicefuls® Juicy Fruit Snacks, 14 count, at Target, Score: 100.0
Offer: L'Oréal Paris Excellence Hair Color at Target, Score: 100.0
Offer: Back to the Roots Raised Bed Gardening Kit with Soil, Seeds and Plant Food, at Target, Score: 100.0
Offer: Colgate® Toothpaste AND Colgate® Toothbrush, select varieties at Walmart or Target, Score: 100.0
Offer: L'Oréal Paris Hair color, select varieties, spend $25 at Target, Score: 1

Enter your search query (type 'exit' to stop): exit
